This file contains the implementation of the RAG system using Langchain

In [9]:
pip install langchain langchain-community langchain-google-genai langchain-chroma python-dotenv pypdf

Note: you may need to restart the kernel to use updated packages.


In [10]:
GOOGLE_API_KEY="yAIzaSyDBN1CVK_9Y3g6vuVaE3R_YpUfEv3uQep8"  # Set your Google API key in .env file

In [11]:
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")  # safer than direct assignment

# Path to your CSV file
csv_dir = "/Users/subashkannan/Desktop/agentic-learning-main/day_2/task_1/data"

# Load CSV documents
documents = []
for filename in os.listdir(csv_dir):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(csv_dir, filename)
        loader = CSVLoader(file_path=file_path)
        documents.extend(loader.load())

# Split the documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)

# Initialize the embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Check if chunks exist
if not chunks:
	print("⚠️ No document chunks found. Please check your CSV file.")
else:
	vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="chroma_db")
	vectordb.persist()
	print("✅ CSV Embeddings created and stored.")


✅ CSV Embeddings created and stored.


/var/folders/rk/z70wbvnj5md0fvz6l6dlpdcw0000gn/T/ipykernel_30702/533978468.py:35: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [12]:
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

load_dotenv()

# Load vector DB
vectordb = Chroma(
    persist_directory="chroma_db",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

# Set up retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Load Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Ask a question
query = "What are the common issues reported in iPhone reviews?"
result = qa_chain.invoke({"query": query})

print("\n🧠 Answer:\n", result["result"])
print("\n📚 Sources:\n", result["source_documents"])



🧠 Answer:
 The reviews mention that performance lags at times.

📚 Sources:
 [Document(id='89e3291c-2408-4e5d-a8c3-435ff3f1fae3', metadata={'source': '/Users/subashkannan/Desktop/agentic-learning-main/day_2/task_1/data/iphone_reviews.csv', 'row': 2}, page_content='product: iPhone 14\nreview_text: Great camera and design, but performance lags at times.\nreviewer_id: rev003\nsentiment: mixed'), Document(id='f5b3a4b1-5195-46ae-95dd-5ca31245015f', metadata={'source': '/Users/subashkannan/Desktop/agentic-learning-main/day_2/task_1/data/iphone_reviews.csv', 'row': 2}, page_content='product: iPhone 14\nreview_text: Great camera and design, but performance lags at times.\nreviewer_id: rev003\nsentiment: mixed'), Document(id='2cb95810-0042-461b-8605-28ce02a76c7c', metadata={'row': 2, 'source': '/Users/subashkannan/Desktop/agentic-learning-main/day_2/task_1/data/iphone_reviews.csv'}, page_content='product: iPhone 14\nreview_text: Great camera and design, but performance lags at times.\nreviewer_